# Web scraping using BeautifulSoup

In [39]:
#import required libraries
import pandas as pd
from bs4 import BeautifulSoup
import requests
import time

In [2]:
#url of web site
url= 'https://editorial.rottentomatoes.com/guide/essential-movies-to-watch-now/'

In [3]:
pages = [1,2]
for i in pages:
    print(pages.index(i)+1)

1
2


In [5]:
#get movies list
movie_list=[]
for page in range(len(pages)):
    r=requests.get(url+str(page+1)+"/").content
    soup=BeautifulSoup(r)
    movies=soup.find_all('div', class_='row countdown-item')
    for movie in movies:
        movie_list.append(movie.find('a').get('href'))

In [25]:
#check
print(movie_list[2])

https://www.rottentomatoes.com/m/400_blows


## Test Cells for Extracting Data for Movie Pages

Watch the steps from [here](https://www.youtube.com/watch?v=YkRLYjWozk8&list=PLzmVhOP_HTKOns2SYCwD3y2jKWRBxvfQd&index=1&t=25s)


In [37]:
#get repsonse for page to build your main code for extarcting
r=requests.get(movie_list[154]).content
soup=BeautifulSoup(r)

In [8]:
movie_name=soup.find('score-board').find('h1').text
rt_score=soup.find('score-board').get('tomatometerscore')
rt_reviews=soup.find('score-board').find('a').text.strip()[:-8]
audinece_score= soup.find('score-board').get('audiencescore')
audiience_reviews= soup.find('score-board').find_all('a')[1].text.strip()[:-9].replace(',',"")
genre= soup.find('score-board').find('p').text.split(',')[1].strip()
release= soup.find('score-board').find('p').text.split(',')[0].strip()
duration=soup.find('score-board').find('p').text.split(',')[2].strip()
d=duration.split(" ")
x=1
if len(duration) == 1:
    time_in_min = int(d[0][:-1])
else:
    time_in_min = int(d[0][:-1])*60 + int(d[1][:-1])
    
movie_info= soup.find('div', class_='movie_synopsis clamp clamp-6 js-clamp').text.strip()
language=soup.find_all('li', class_="meta-row clearfix")[1].find_all('div')[1].text.strip()
director=soup.find_all('li', class_="meta-row clearfix")[2].find_all('div')[1].find('a').text.strip()
producer=soup.find_all('li', class_="meta-row clearfix")[3].contents[3].text.strip().replace("  ", "").replace("\n","")
writer=soup.find_all('li', class_="meta-row clearfix")[4].contents[3].text.strip().replace("  ", "").replace("\n","")

    
print(language)

English


## Loop over all pages after solving all problem in test phase
- If you face any error get the page that cause error 
- Start to test why error happened in test cell
- Add the needed steps to overcome the error and get needed data below

In [40]:
#loop to get all movie page after testing and covering all pages structure to avoide any break on our code
top_300=[]
for movie in movie_list:  
    r=requests.get(movie).content
    soup=BeautifulSoup(r)
    rank=movie_list.index(movie)+1
    movie_name=soup.find('score-board').find('h1').text
    rt_score=soup.find('score-board').get('tomatometerscore')
    rt_reviews=soup.find('score-board').find('a').text.strip()[:-8]
    audinece_score= soup.find('score-board').get('audiencescore')
    audiience_reviews= soup.find('score-board').find_all('a')[1].text.strip()[:-9].replace(',',"")
    genre= soup.find('score-board').find('p').text.split(',')[1].strip()
    release= soup.find('score-board').find('p').text.split(',')[0].strip()
    duration=soup.find('score-board').find('p').text.split(',')[2].strip()
    x=1
    if len(duration) == 1:
        time_in_min = int(d[0][:-1])
    else:
        time_in_min = int(d[0][:-1])*60 + int(d[1][:-1])
    
    movie_info= soup.find('div', class_='movie_synopsis clamp clamp-6 js-clamp').text.strip()


    rating, language, director, producer=None, None, None, None
    for element in soup.find_all('li', class_="meta-row clearfix"):
        if element.find('div', class_='meta-label subtle').text == 'Rating:':
            rating = element.find('div', class_='meta-value').text.strip().replace('\n',"")
        if element.find('div', class_='meta-label subtle').text == 'Original Language:':
            language = element.find('div', class_='meta-value').text.strip().replace('\n',"")
        if element.find('div', class_='meta-label subtle').text == 'Director:':
            director = element.find('div', class_='meta-value').text.strip().replace('\n',"")
        if element.find('div', class_='meta-label subtle').text == 'Producer:':
            producer = element.find('div', class_='meta-value').text.strip().replace('\n',"")
        
    top_300.append({
                    'rank':rank,
                    'name': movie_name,
                    'critics_score' : int(rt_score),
                    'critics_review_numbers' : int(rt_reviews),
                    'audience_score' : int(audinece_score),
                    'audience_reviews' :(audiience_reviews),
                    'year' : int(release),
                    'genre' : genre,
                    'rating': rating,
                    'duration' : duration,
                    'time_in_minutes' : time_in_min,
                    'movie_page' : movie,
                    'language': language,
                    'producer':producer,
                    'writer': writer
                    
                    })
    if int(rank) % 23 = 0:
        time.sleep(1)
    
df=pd.DataFrame(top_300, columns=['rank','name','critics_score','critics_review_numbers','audience_score', 'audience_reviews','year',
                    'genre', 'rating', 'duration','time_in_minutes', 'movie_page', 'language', 'producer', 'writer'])
df.head()

,rank,name,critics_score,critics_review_numbers,audience_score,audience_reviews,year,genre,rating,duration,time_in_minutes,movie_page,language,producer,writer
0,1,12 Angry Men,100,60,97,100000,1957,Drama,None,1h 35m,95,https://www.rottentomatoes.com/m/1000013_12_an...,English,"Henry Fonda, ...",Reginald Rose
1,2,2001: A Space Odyssey,91,117,89,250000,1968,Sci-fi,G,2h 19m,95,https://www.rottentomatoes.com/m/2001_a_space_...,English,Stanley Kubrick,Reginald Rose
2,3,The 400 Blows,99,71,94,25000,1959,Crime/Drama,None,1h 33m,95,https://www.rottentomatoes.com/m/400_blows,French (France),François Truffaut,Reginald Rose
3,4,"The Adventures of Priscilla, Queen of the Desert",94,48,88,25000,1994,Comedy/Lgbtq+,R,1h 42m,95,https://www.rottentomatoes.com/m/adventures_of...,English,"Al Clark, ...",Reginald Rose
4,5,The Adventures of Robin Hood,100,50,89,25000,1938,Adventure/Action,PG,1h 42m,95,https://www.rottentomatoes.com/m/1000355-adven...,English,Hal B. Wallis,Reginald Rose


In [46]:
df.sample(10)

,rank,name,critics_score,critics_review_numbers,audience_score,audience_reviews,year,genre,rating,duration,time_in_minutes,movie_page,language,producer,writer
148,149,Jaws,97,99,90,250000,1975,Mystery & thriller/Horror,PG,2h 4m,95,https://www.rottentomatoes.com/m/jaws,English,"David Brown, ...",Reginald Rose
204,205,The Passion of Joan of Arc,98,60,93,10000,1928,History/Drama,None,1h 17m,95,https://www.rottentomatoes.com/m/passion_of_jo...,None,None,Reginald Rose
181,182,Moonlight,98,397,79,25000,2016,Drama/Lgbtq+,R (Drug Use|Brief Violence|Language Throughout...,1h 51m,95,https://www.rottentomatoes.com/m/moonlight_2016,English,"Adele Romanski, ...",Reginald Rose
13,14,Almost Famous,89,178,92,250000,2000,Comedy/Drama,R (Drug Content|Brief Nudity|Language),2h 2m,95,https://www.rottentomatoes.com/m/almost_famous,English,"Ian Bryce, ...",Reginald Rose
278,279,To Kill a Mockingbird,93,68,93,100000,1962,Drama,None,2h 9m,95,https://www.rottentomatoes.com/m/to_kill_a_moc...,English,Alan J. Pakula,Reginald Rose
299,300,Zero Dark Thirty,91,309,80,100000,2012,Drama/Mystery & thriller,R (Language|Brutal Disturbing Images|Strong Vi...,2h 37m,95,https://www.rottentomatoes.com/m/zero_dark_thirty,English,"Mark Boal, ...",Reginald Rose
80,81,Dracula,94,51,82,25000,1931,Horror,None,1h 15m,95,https://www.rottentomatoes.com/m/1006234-dracula,English,"Tod Browning, ...",Reginald Rose
16,17,Amour,93,227,82,10000,2012,Drama,PG-13 (Brief Language|A Disturbing Act|Mature ...,2h 7m,95,https://www.rottentomatoes.com/m/amour_2013,French (France),"Margaret Ménégoz, ...",Reginald Rose
138,139,Ikiru,98,53,97,10000,1952,Drama/Lgbtq+,None,2h 23m,95,https://www.rottentomatoes.com/m/ikiru,Japanese,Sojiro Motoki,Reginald Rose
276,277,Titanic,88,250,69,250000,1997,History/Drama,PG-13 (Disaster Related Peril|Brief Language|N...,3h 15m,95,https://www.rottentomatoes.com/m/titanic,English,"James Cameron, ...",Reginald Rose
